# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [15]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

# ✅ Load environment variables
load_dotenv(override=True)
api_key = os.getenv("GOOGLE_GEMINI_API_KEY")

# ✅ Check API Key
if api_key and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please check your .env file!")

# ✅ Configure Google Gemini API
genai.configure(api_key=api_key)

# ✅ Set Gemini Model
MODEL = "models/gemini-1.5-pro-latest"
print(f"Using model: {MODEL}")


API key looks good so far
Using model: models/gemini-1.5-pro-latest


In [20]:
import os
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
from dotenv import load_dotenv

# ✅ Load environment variables
load_dotenv()
api_key = os.getenv("GOOGLE_GEMINI_API_KEY")

if not api_key:
    raise ValueError("⚠️ Missing Google Gemini API Key. Set GOOGLE_GEMINI_API_KEY in your .env file!")

# ✅ Configure Google Gemini API
genai.configure(api_key=api_key)

# ✅ Set request headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """A utility class to represent a Website that we have scraped, now with links."""

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # ✅ Handle HTTP errors properly
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

# ✅ Summarization using Gemini
def summarize_website(url):
    website = Website(url)
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")

    # ✅ Fix: Pass a single string, NOT a list
    response = model.generate_content(f"Summarize this webpage:\n{website.get_contents()}")

    # ✅ Correctly access the response text
    return response.text if hasattr(response, "text") else response.candidates[0].content

# ✅ Example usage
summary = summarize_website("https://techseva-it-solutions.vercel.app/")
print(summary)


TechSeva Solutions is an IT solutions agency that helps businesses grow using cutting-edge technology and custom digital solutions.  They offer a wide range of services, including website and app development, UI/UX design, branding, digital marketing, business automation, video editing, and IT consulting.  They emphasize data-driven strategies, AI-powered solutions, and 24/7 support.  Their website highlights their commitment to quality, security, and user experience, catering to startups, e-commerce businesses, and other organizations.  They boast 50 satisfied clients, 500 launched projects, and 5 years of experience.  Contact information includes phone numbers, email addresses, and a physical address in Madhubani, Bihar, India.  Their blog covers topics such as tech trends, UI/UX design, business automation, and AI's role in digital marketing.



In [21]:
ed = Website("https://techseva-it-solutions.vercel.app/")
ed.links

['#',
 '#',
 '#home',
 '#about',
 '#services',
 '#features',
 '#blog',
 '#services2',
 '#',
 '#about',
 'https://wa.me/919470074183',
 'https://www.instagram.com/techseva_solutions?igsh=MXcxM2xjY3dlcnlodg==',
 'https://facebook.com/your-page',
 'https://www.linkedin.com/in/abhishek-yadav-70a69829a/',
 'mailto:info@techsevasolutions.com',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 'tel:+91-6200087830',
 'tel:+91-9470074183',
 'mailto:info@desinic.com',
 'mailto:help@desinic.com',
 '#',
 '#top']

## First step: Have fix with models/gemini-1.5-pro-latest figure out which links are relevant

### Use a call to gemini-1.5-pro-latest to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [26]:
link_system_prompt = """You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company, 
such as links to an About page, a Company page, or Careers/Jobs pages.

You should respond in JSON format as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""


In [27]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. 
You are able to decide which of the links would be most relevant to include in a brochure about the company, 
such as links to an About page, a Company page, or Careers/Jobs pages.

You should respond in JSON format as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [28]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [29]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://techseva-it-solutions.vercel.app/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#
#
#home
#about
#services
#features
#blog
#services2
#
#about
https://wa.me/919470074183
https://www.instagram.com/techseva_solutions?igsh=MXcxM2xjY3dlcnlodg==
https://facebook.com/your-page
https://www.linkedin.com/in/abhishek-yadav-70a69829a/
mailto:info@techsevasolutions.com
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
tel:+91-6200087830
tel:+91-9470074183
mailto:info@desinic.com
mailto:help@desinic.com
#
#top


In [33]:
import google.generativeai as genai
import json

# ✅ Configure Google Gemini API
genai.configure(api_key="GOOGLE_GEMINI_API_KEY")

def get_links(url):
    website = Website(url)
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")
    
    # ✅ Ensure proper input formatting for Gemini
    response = model.generate_content([
        {"role": "system", "content": link_system_prompt},
        {"role": "user", "content": get_links_user_prompt(website)}
    ])

    # ✅ Extract response content and parse JSON
    result = response.text if response and hasattr(response, "text") else "{}"
    return json.loads(result) if result else {}


In [53]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/sesame/csm-1b',
 '/mistralai/Mistral-Small-3.1-24B-Instruct-2503',
 '/google/gemma-3-27b-it',
 '/ds4sd/SmolDocling-256M-preview',
 '/Qwen/QwQ-32B',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/jasperai/LBM_relighting',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/prs-eth/thera',
 '/spaces/Qwen/QwQ-32B-Demo',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/open-r1/codeforces-cots',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/facebook/natural_reasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',

In [54]:
import google.generativeai as genai
import json
import requests
from bs4 import BeautifulSoup

# ✅ Configure Google Gemini API
genai.configure(api_key="GOOGLE_GEMINI_API_KEY")

# ✅ Define request headers
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# ✅ Website Scraping Class
class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, "html.parser")
        self.title = soup.title.string if soup.title else "No title found"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        self.links = [link.get("href") for link in soup.find_all("a") if link.get("href")]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

# ✅ System Prompt
link_system_prompt = """
You are provided with a list of links found on a webpage.
You must decide which links would be most relevant to include in a brochure about the company.
Focus on links such as 'About', 'Company', 'Careers', 'Jobs', or similar.
Return the response ONLY in JSON format as shown below:

{
    "links": [
        {"type": "about page", "url": "https://full.url/about"},
        {"type": "careers page", "url": "https://full.url/careers"}
    ]
}
"""

# ✅ Function to Get Relevant Links
def get_links(url):
    website = Website(url)
    model = genai.GenerativeModel("models/gemini-1.5-pro-latest")

    # ✅ Ensure the input format is a single string
    prompt = f"{link_system_prompt}\n\nExtract relevant links from this webpage:\n\n{website.get_contents()}"

    response = model.generate_content(prompt)

    # ✅ Handle Gemini Response Correctly
    if response and hasattr(response, "text"):
        raw_json = response.text
    elif response and hasattr(response, "candidates") and response.candidates:
        raw_json = response.candidates[0].content
    else:
        return {"error": "No valid response from Gemini"}

    # ✅ Safely Convert JSON Response
    try:
        return json.loads(raw_json)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON format returned by Gemini", "raw_response": raw_json}

# ✅ Test the function
print(get_links("https://huggingface.co"))


InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

In [47]:
get_links("https://huggingface.co")

KeyError: "Unable to determine the intended type of the `dict`. For `Content`, a 'parts' key is expected. For `Part`, either an 'inline_data' or a 'text' key is expected. For `Blob`, both 'mime_type' and 'data' keys are expected. However, the provided dictionary has the following keys: ['role', 'content']"

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [41]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [42]:
print(get_all_details("https://huggingface.co"))

InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

In [ ]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>